In [1]:
import astropy.io.fits as fits
import des_functions
import treecorr
import numpy as np
import matplotlib.pyplot as plt
from more_itertools import locate
import time
import pyccl as ccl

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['cmr10']
plt.rcParams['mathtext.fontset'] ='cm'
plt.rc('axes', unicode_minus=False)

In [8]:
data_dir = '/home/b7009348/WGL_project/DES-data/'
lens_file = data_dir+'DES_Y1A1_Lenses_z=0.3-0.45.fits'
rand_file = data_dir+'DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'
im3_file = data_dir+'y1_im3_shapes_matched.fits'
mcal_file = data_dir+'y1_mcal_shapes_matched.fits'
z_file = data_dir+'mcal_BPZ_(binning).fits'

In [9]:
sep_bins = 25
theta_min = 2.5
theta_max = 250

### No patches 

In [16]:
with fits.open(lens_file) as hdu:
    data = hdu[1].data
    ra_l = data['RA']
    dec_l = data['DEC']
    w_l = data['weight']

lens_shot = treecorr.Catalog(ra=ra_l, dec=dec_l, w=w_l, ra_units='deg', dec_units='deg')

del data, ra_l, dec_l, w_l

with fits.open(rand_file) as hdu:
    data = hdu[1].data
    indexes = list(locate(data['z'], lambda x: 0.3 < x < 0.45))
    ra_r = data['RA'][indexes]
    dec_r = data['DEC'][indexes]
    rand_z = data['z'][indexes]

rand_shot = treecorr.Catalog(ra=ra_r, dec=dec_r, ra_units='deg', dec_units='deg')

del data, indexes, ra_r, dec_r, rand_z

with fits.open(mcal_file) as hdu:
    data = hdu[1].data
    ra_s = data['ra']
    dec_s = data['dec']
    e1 = data['e1']
    e2 = data['e2']
    R11 = data['R11']

mcal_shot = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', g1=e1, g2=e2)

Rsp = np.mean(R11)

del data, ra_s, dec_s, e1, e2, R11

print('Patchless catalogues created.')

## Patches

In [17]:
with fits.open(lens_file) as hdu:
    data = hdu[1].data
    ra_l = data['RA']
    dec_l = data['DEC']
    w_l = data['weight']

lens_patches = treecorr.Catalog(ra=ra_l, dec=dec_l, w=w_l, ra_units='deg', dec_units='deg', patch_centers=data_dir+'jackknife_patch_centers')

del data, ra_l, dec_l, w_l

with fits.open(rand_file) as hdu:
    data = hdu[1].data
    indexes = list(locate(data['z'], lambda x: 0.3 < x < 0.45))
    ra_r = data['RA'][indexes]
    dec_r = data['DEC'][indexes]
    rand_z = data['z'][indexes]

rand_patches = treecorr.Catalog(ra=ra_r, dec=dec_r, ra_units='deg', dec_units='deg', patch_centers=data_dir+'jackknife_patch_centers')

del data, indexes, ra_r, dec_r, rand_z

with fits.open(mcal_file) as hdu:
    data = hdu[1].data
    ra_s = data['ra']
    dec_s = data['dec']
    e1 = data['e1']
    e2 = data['e2']
    R11 = data['R11']

mcal_patches = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', g1=e1, g2=e2, patch_centers=data_dir+'jackknife_patch_centers')

del data, ra_s, dec_s, e1, e2, R11

print('Patched catalogues created.')

In [41]:
def precision_test(bin_slop, sep_bins):
    
    start = time.time()
    
    # preallocate output arrays
    shot_gammat = np.zeros([sep_bins])
    shot_theta = np.zeros([sep_bins])
    
    print('Starting patchless correlation...')
    
    # calculate tangential shear with lenses
    ng = treecorr.NGCorrelation(nbins=sep_bins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin', verbose=1, bin_slop=bin_slop)
    ng.process(lens_shot, mcal_shot)

    # calculate tangential shear with randoms
    rg = treecorr.NGCorrelation(nbins=sep_bins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin', bin_slop=bin_slop)
    rg.process(rand_shot, mcal_shot)

    xi_l = ng.xi
    xi_r = rg.xi

    shot_gammat = 1.0/Rsp * (xi_l - xi_r)
    shot_theta = np.exp(ng.meanlogr)

    print(shot_gammat)

    # preallocate output arrays
    patches_gammat = np.zeros([sep_bins])
    patches_theta = np.zeros([sep_bins])
    
    print('\n Starting patched correlation...')

    # calculate tangential shear with lenses
    ng = treecorr.NGCorrelation(nbins=sep_bins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin', verbose=1, bin_slop=bin_slop)
    ng.process(lens_patches, mcal_patches)

    # calculate tangential shear with randoms
    rg = treecorr.NGCorrelation(nbins=sep_bins, min_sep=theta_min, max_sep=theta_max, sep_units='arcmin', bin_slop=bin_slop)
    rg.process(rand_patches, mcal_patches)

    xi_l = ng.xi
    xi_r = rg.xi

    patches_gammat = 1.0/Rsp * (xi_l - xi_r)
    patches_theta = np.exp(ng.meanlogr)

    print(patches_gammat)
    
    end = time.time()
    runtime = end-start
    print('\n %f \n' %runtime)
    
    return shot_gammat, patches_gammat

In [51]:
bin_slops = 0.4342944819032518
sep_bins = 20

diff = np.zeros(sep_bins)

shot, patch = precision_test(bin_slops, sep_bins)

diff = abs(shot - patch)

Starting patchless correlation...
[1.22343045e-03 1.08374556e-03 7.50357134e-04 7.14348122e-04
 5.20163432e-04 3.43916069e-04 2.91516673e-04 2.39981298e-04
 1.88691239e-04 1.72773452e-04 1.40410142e-04 1.13359805e-04
 1.18959892e-04 7.54907090e-05 7.50158509e-05 6.99658536e-05
 5.70959299e-05 4.54247436e-05 4.35179537e-05 4.19731572e-05]

 Starting patched correlation...
[1.22850312e-03 1.08099893e-03 7.90436868e-04 6.62166370e-04
 5.24727494e-04 3.87596783e-04 2.75520094e-04 2.30092429e-04
 1.95619391e-04 1.68357967e-04 1.36384474e-04 1.16712450e-04
 1.17757279e-04 8.40498908e-05 7.41308276e-05 6.92606510e-05
 5.73138861e-05 4.53669400e-05 4.06265932e-05 4.57972790e-05]

 108.148089 



In [53]:
real_values = np.load(data_dir+'ia_full_values.npz')
print(real_values['IA'])
print(diff)

[ 9.43802183e-05  1.86657550e-03  1.40719781e-03 -2.55295914e-04
 -6.51078079e-05  6.53064049e-04  4.29204973e-04  9.94408327e-05
  5.22798168e-04  3.78934484e-04  6.18269575e-05 -5.56187781e-05
  1.56824562e-04  1.89447451e-04  7.47752518e-05  9.87335743e-05
  1.62952739e-04  1.07624378e-04  5.55231532e-05  6.45335103e-05]
[5.07266602e-06 2.74662853e-06 4.00797342e-05 5.21817524e-05
 4.56406224e-06 4.36807136e-05 1.59965788e-05 9.88886917e-06
 6.92815149e-06 4.41548577e-06 4.02566793e-06 3.35264488e-06
 1.20261373e-06 8.55918172e-06 8.85023290e-07 7.05202622e-07
 2.17956185e-07 5.78036133e-08 2.89136048e-06 3.82412186e-06]


In [54]:
import glob 
files  = glob.glob(data_dir+'gold_catalogues/*.fits')
print(len(files))

682
